In [1]:
import pandas as pd
import random
import numpy as np
import string
import re
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('D:/personal project/twitter_sentiment_analysis/datasets/tweet.csv')
del df[df.columns[0]]
df

,sentimen,tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...
1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [7]:
df.sentimen.value_counts()

sentimen
positif    612
netral     607
negatif    596
Name: count, dtype: int64

# Cleaning

In [8]:
tweets_cleaned = []

  # Looping setiap kalimat
for sentence in df.tweet:

    sentence = re.sub(r"(pic.twitter.com/[^\s]+)", '', sentence) # remove twitter image link

    sentence = re.sub(r'http[s]?\://\S+', "", sentence) # remove url

    sentence = re.sub(r'\S*.com.*\S', "", sentence)

    sentence = re.sub(r'\S*.co.id.*\S', "", sentence)

    sentence = re.sub(r'([a-z])([A-Z])', r'\1 \2', sentence) # remove kata kata berdempet

    sentence = re.sub(r"#\w+", "", sentence) #remove hashtag

    sentence = sentence.replace('@', '') # remove at@

    sentence = re.sub(r'\b\d+\b', '', sentence) # remove angka berdempet dg kata

    sentence = re.sub('\d', '', sentence) # remove angka

    sentence = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence) # Remove reapeating char

    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence) # remove punctuation and special character

    sentence = re.sub(r"\s+", " ", sentence) # remove double space

    sentence = sentence.lower() # case folding

    sentence = sentence.strip()

    tweets_cleaned.append(sentence)

In [9]:
display(df.tweet[0])
display(tweets_cleaned[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

'kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung'

# Spell Correcting

In [11]:
spell_dictionary = open('D:/personal project/twitter_sentiment_analysis/datasets/slangword.txt').read()
spell_dictionary

'&:dan\n+:tambah\n/:atau\n=:sama dengan\nababil:anak ingusan\nabal2:palsu\nabal:palsu\nad:ada\nakooh:aku\nalay:norak\nalbm:album\nampe:sampai\nanjir:waw\nanyway:ngomong-ngomong\naq:aku\nasap:secepatnya\nato:atau\natw:atau\nava:foto profil\nbaget:keras kepala\nbaper:bawa perasaan\nbapuk:rusak\nbcr:bicara\nbebeb:pacar\nbegin:awal\nbejibun:bertumpuk banyak\nbener:benar\nber2:berdua\nber3:bertiga\nbetter:lebih baik\nbf:pacar\nbgt:banget\nbhas:bahas\nbhg:bahagia\nbikin:buat\nbinggo:banget\nbingit:banget\nbklan:bakalan\nbkn:bukan\nblg:bilang\nbnr:benar\nbnyk:banyak\nbr:baru\nbrb:akan segera kembali\nbrjaya:berjaya\nbs:bisa\nbsk:besok\nbtw:ngomong-ngomong\ncm:cuma\ncmiiw:koreksi saya\ncnt:cinta\ncongrats:selamat\ncrop:potong\ncya:sampai jumpa lagi\nd:di\nda:ada\ndafuk:kurang ajar\ndamat:tidak peduli\ndempa:demi apa\ndgn:dengan\ndkt:dekat\ndlm:dalam\ndlu:dulu\ndmn:dimana\ndpt:dapat\ndr:dari\ndrpd:daripada\ndsna:disana\ndugem:dunia gemerlap\nelo:kamu\nelu:kamu\nemg:memang\nepsd:episode\nfake:fa

In [22]:
def create_spell_check_dictionary(spell_check_data):
    spell_check_dict = {}
    lines = spell_check_data.strip().split("\n")
    for line in lines:
        key, value = line.split(":")
        spell_check_dict[key] = value
    return spell_check_dict

In [23]:
spell_dictionary = create_spell_check_dictionary(spell_dictionary)
spell_dictionary

{'&': 'dan',
 '+': 'tambah',
 '/': 'atau',
 '=': 'sama dengan',
 'ababil': 'anak ingusan',
 'abal2': 'palsu',
 'abal': 'palsu',
 'ad': 'ada',
 'akooh': 'aku',
 'alay': 'norak',
 'albm': 'album',
 'ampe': 'sampai',
 'anjir': 'waw',
 'anyway': 'ngomong-ngomong',
 'aq': 'aku',
 'asap': 'secepatnya',
 'ato': 'atau',
 'atw': 'atau',
 'ava': 'foto profil',
 'baget': 'keras kepala',
 'baper': 'bawa perasaan',
 'bapuk': 'rusak',
 'bcr': 'bicara',
 'bebeb': 'pacar',
 'begin': 'awal',
 'bejibun': 'bertumpuk banyak',
 'bener': 'benar',
 'ber2': 'berdua',
 'ber3': 'bertiga',
 'better': 'lebih baik',
 'bf': 'pacar',
 'bgt': 'banget',
 'bhas': 'bahas',
 'bhg': 'bahagia',
 'bikin': 'buat',
 'binggo': 'banget',
 'bingit': 'banget',
 'bklan': 'bakalan',
 'bkn': 'bukan',
 'blg': 'bilang',
 'bnr': 'benar',
 'bnyk': 'banyak',
 'br': 'baru',
 'brb': 'akan segera kembali',
 'brjaya': 'berjaya',
 'bs': 'bisa',
 'bsk': 'besok',
 'btw': 'ngomong-ngomong',
 'cm': 'cuma',
 'cmiiw': 'koreksi saya',
 'cnt': 'cinta

In [24]:
# def tweet_spell_correction(tweet, spell_dictionary):
#     words = tweet.split()

#     corrected_words = []

#     for word in words:
#         if word in spell_dictionary:
#             corrected_word = spell_dictionary[word]
#             corrected_words.append(corrected_word)
#         else:
#             corrected_words.append(word)

#     corrected_tweet = ' '.join(corrected_words)

#     return corrected_tweet

def tweet_spell_correction(tweet, spell_dictionary):
    words = tweet.split()

    corrected_words = []

    for word in words:
        if word in spell_dictionary:
            corrected_word = spell_dictionary[word]
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    corrected_tweet = ' '.join(corrected_words)

    return corrected_tweet

In [25]:
tweet_spell_corrected = []

for idx in range(len(tweets_cleaned)):
    corrected = tweet_spell_correction(tweets_cleaned[idx], spell_dictionary)
    tweet_spell_corrected.append(corrected)

display(tweets_cleaned[:5])
print('--'*50)
display(tweet_spell_corrected[:5])

['kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti hoax buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik maju lagi menang total debat',
 'bagi sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat',
 'negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap negara itu pasti ngutang utang bisa dibayar kalo negara dapet penghasilan penghasilan negara itu ya dari pajak']

----------------------------------------------------------------------------------------------------


['kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti palsu buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik maju lagi menang total debat',
 'bagi sumatera selatan asian games berdampak pada ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat',
 'negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap negara itu pasti ngutang utang bisa dibayar kalo negara dapet penghasilan penghasilan negara itu ya dari pajak']

In [26]:
display(tweets_cleaned[338]) # 266,
print('--'*50)
display(tweet_spell_corrected[338])

'aku memang setuju macro ekonominya jokowi jago bgt meski mungkin itu memang rancangan yg sudah ditulis sm ahli yg merancang pembangunan indonesia ya tapi jokowi memang memahami dg baik'

----------------------------------------------------------------------------------------------------


'aku memang setuju macro ekonominya jokowi jago banget meski mungkin itu memang rancangan yang sudah ditulis sama ahli yang merancang pembangunan indonesia ya tapi jokowi memang memahami dengan baik'

# Stopword Removal

In [27]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
# stopwords_id = ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'seharusnya', 'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 'saja', 'toh', 'walau', 'tolong', 'tentu', 'amat', 'apalagi', 'bagaimanapun']

custom_stopword = ['ah', 'yang']

In [29]:
def remove_stopwords_indo(text):
    # Kode untuk menghapus stopwords
    text_lower = text.lower()
    stop_words = set(custom_stopword) # chanfe stopwords_id
    tokens = word_tokenize(text_lower)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [30]:
tweet_no_stopwords = []

for idx in range(len(tweet_spell_corrected)):
    cleaned = remove_stopwords_indo(tweet_spell_corrected[idx])
    tweet_no_stopwords.append(cleaned)

display(tweet_spell_corrected[248])
display(tweet_no_stopwords[248])

'dipikirnya pengembangan esport itu main doang aplikasi yang ada di smartphone itu banyak yang mulainya dari main doang terus jadi aplikasi yang banyak manfaatnya kata siapa pengangguran digaji banyak baca buka wawasan jangan mpermalukan generasi z'

'dipikirnya pengembangan esport itu main doang aplikasi ada di smartphone itu banyak mulainya dari main doang terus jadi aplikasi banyak manfaatnya kata siapa pengangguran digaji banyak baca buka wawasan jangan mpermalukan generasi z'

# SKIP STEMMING

# Stemming

In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
# buat fungsi untuk stemming
def stemming_sastrawi(text):
  # define penggunaan sastrawi sebagai stemmer
  stemmer = StemmerFactory().create_stemmer()

  # pakai stemmer
  cleaned_text = stemmer.stem(text)
  return cleaned_text

In [ ]:
tweet_stemmed = []

# for index in range(len(tweet_no_stopwords)):
#   stemmed = stemming_sastrawi(tweet_no_stopwords[index])
#   tweet_stemmed.append(stemmed)

In [ ]:
display(tweet_no_stopwords[40])
display(tweet_stemmed[40])

'orang luar negeri sangat menghargai prestasi dilakukan jokowi dinegaranya sendiri banyak mencaci'

'orang luar negeri sangat harga prestasi laku jokowi negara sendiri banyak caci'

In [ ]:
new_tweet_stemmed = pd.DataFrame(tweet_stemmed, columns=['tweet'])
new_tweet_stemmed.to_csv('/content/drive/MyDrive/sample_dataset/new_tweet_stemmed.csv')

# Texts Backtranslation Augmentation

In [ ]:
!pip install googletrans

In [ ]:
from googletrans import Translator

def translate_texts(texts, target_language):
    translator = Translator()

    translated_texts = []

    for text in texts:
        translation = translator.translate(text, dest=target_language)
        translated_texts.append(translation.text)

    return translated_texts

# Translator().translate()
# Example usage:
# texts_to_translate = ["Halo, apa kabar?", "Saya suka bahasa Python."]
# translated_texts = translate_texts(texts_to_translate, target_language='en')

# for original, translated in zip(texts_to_translate, translated_texts):
#     print(f"Original: {original} - Translated: {translated}")


In [ ]:
tweet_en_trans = []

# for idx in range(len(tweet_stemmed)):
#   translated = translate_texts(tweet_stemmed[idx], target_lenguage='en')
#   tweet_en_trans.append(translated)

tweet_en_trans = translate_texts(tweet_stemmed, target_language='en')
# tweet_en_trans

TypeError: ignored

In [ ]:
display(tweet_stemmed[1])
display(tweet_en_trans[1])

In [ ]:
tweet_id_trans = []

for idx in range(len(tweet_en_trans)):
  translated = translate_texts(tweet_en_trans[idx], target_lenguage='id')
  tweet_id_trans.append(translated)

In [ ]:
display(tweet_stemmed[1])
display(tweet_en_trans[1])
display(tweet_id_trans[1])

# Synonim Replacment

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00


In [31]:
import random
import sentencepiece
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [32]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...


True

In [33]:
# Load the T5 model and tokenizer for style transfer and text paraphrasing
model_style_transfer = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer_style_transfer = T5Tokenizer.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
# Function for synonym replacement using WordNet
def synonym_replacement(text, probability=0.1):
    words = word_tokenize(text)
    augmented_words = []

    for word in words:
        if random.uniform(0, 1) < probability:
            synonyms = wordnet.synsets(word, lang='ind')
            if synonyms:
                synonym = synonyms[0].lemmas(lang='ind')[0].name()
                augmented_words.append(synonym)
            else:
                augmented_words.append(word)
        else:
            augmented_words.append(word)

    return ' '.join(augmented_words)

def style_transfer(text, model, tokenizer):
    # Specify the desired style transfer task using a prefix
    input_text = "change the style: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the style-transferred text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the transformed text
    style_transferred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return style_transferred_text

def text_paraphrasing(text, model, tokenizer):
    # Specify the text paraphrasing task using a prefix
    input_text = "paraphrase: " + text

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the paraphrased text
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)

    # Decode and return the paraphrased text
    paraphrased_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return paraphrased_text

def sentence_shuffling(text, probability=0.1):
    sentences = sent_tokenize(text)

    if random.uniform(0, 1) < probability:
        random.shuffle(sentences)
        return ' '.join(sentences)
    else:
        return text

def sentence_deletion(text, probability=0.1):
    sentences = sent_tokenize(text)

    if random.uniform(0, 1) < probability and len(sentences) > 1:
        # Delete a random sentence
        sentences.pop(random.randint(0, len(sentences) - 1))
        return ' '.join(sentences)
    else:
        return text

def word_insertion(text, probability=0.1):
    words = word_tokenize(text)

    if random.uniform(0, 1) < probability:
        # Insert a random word at a random position
        word_to_insert = "merdeka"
        position_to_insert = random.randint(0, len(words))
        words.insert(position_to_insert, word_to_insert)
        return ' '.join(words)
    else:
        return text

# Load the T5 model and tokenizer for style transfer and text paraphrasing
# model_style_transfer = T5ForConditionalGeneration.from_pretrained("t5-small")
# tokenizer_style_transfer = T5Tokenizer.from_pretrained("t5-small")

# Function to perform text augmentation using various techniques
def augment_text_list(texts, augmentation_functions, probabilities):
    augmented_texts = []

    for text in texts:
        augmented_text = text

        for func, prob in zip(augmentation_functions, probabilities):
            augmented_text = func(augmented_text, probability=prob)

        augmented_texts.append(augmented_text)

    return augmented_texts

In [36]:
tweet_augmentation = []

# Define augmentation functions and their corresponding probabilities
augmentation_functions = [synonym_replacement, text_paraphrasing, style_transfer,]
probabilities = [0.5,]
# tweet_synm_replc = synonym_replacement(tweet_stemmed)
tweet_augmentation = augment_text_list(tweet_no_stopwords, augmentation_functions, probabilities)

In [38]:
display(tweet_no_stopwords[:10])
print('-'*50)
display(tweet_augmentation[:10])

['kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti palsu buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik maju lagi menang total debat',
 'bagi sumatera selatan asian games berdampak pada ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat',
 'negara kita ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap negara itu pasti ngutang utang bisa dibayar kalo negara dapet penghasilan penghasilan negara itu ya dari pajak',
 'bisikin bapak jokowi cuma mikirin perputaran duit di golongan tertentu esports hanya sebagian kecil dari sekian besar penentu pertumbuhan perekonomian indonesia bukan hal fundamental betul dikatakan prabowo walau dengan bahas sederhana tapi mrpkan aspek fundamental ekonomi',
 'masa tenang masih ngoceh ajattp jokowi harga mati',
 'keji balas di tps kerjasa ekonomi bilat

--------------------------------------------------


['bercakap prabowo indonesia belum dihargai bangsa asing berita ini jelas imitatif buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi bakat dari habib luthfi seharga mercy',
 'di era jokowi ekonomi Indonesia semakin baik ke_depan lagi menang total debat',
 'bagi sumatera selatan asian games berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia ke_depan jokowi hebat',
 'negara kita ngutang buat bngun asas udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap keadaan itu pasti ngutang utang bisa dibayar kalo keadaan dapet penghasilan penghasilan negara itu bahwasanya dari pajak',
 'bisikin bapak jokowi cuma mikirin perputaran duit atas golongan pasti esports cuma sebagian kecil dari sekian besar penentu kemajuan perekonomian indonesia belum hal fundamental benar dikatakan prabowo meskipun dengan bahas baru tapi mrpkan aspek asas ekonomi',
 'masa anteng baru ngoceh ajattp jokowi harga mati',
 'bengis balas atas tps kerjasa ekonomi b

In [39]:
display(tweet_no_stopwords[10])
print('-'*50)
display(tweet_augmentation[10])

'orang baik jelas islamnya baju putih pilih pemimpin itu sudah jelas banyak pencapaiannya nah ini dia pencapaian ekonomi selama pemerintahan jokowi coblos yuk kerja pasti'

--------------------------------------------------


'orang baik jelas islamnya baju beruban ambil pemimpin itu pernah jelas banyak pencapaiannya nah ini dia kejayaan ekonomi selama pemerintahan jokowi coblos yuk bekerja pasti'

In [40]:
tweet_augmentation

['bercakap prabowo indonesia belum dihargai bangsa asing berita ini jelas imitatif buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi bakat dari habib luthfi seharga mercy',
 'di era jokowi ekonomi Indonesia semakin baik ke_depan lagi menang total debat',
 'bagi sumatera selatan asian games berdampak atas ekonomi langsung diprediksi berhasil triliun Indonesia ke_depan jokowi hebat',
 'negara kita ngutang buat bngun asas udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap keadaan itu pasti ngutang utang bisa dibayar kalo keadaan dapet penghasilan penghasilan negara itu bahwasanya dari pajak',
 'bisikin bapak jokowi cuma mikirin perputaran duit atas golongan pasti esports cuma sebagian kecil dari sekian besar penentu kemajuan perekonomian indonesia belum hal fundamental benar dikatakan prabowo meskipun dengan bahas baru tapi mrpkan aspek asas ekonomi',
 'masa anteng baru ngoceh ajattp jokowi harga mati',
 'bengis balas atas tps kerjasa ekonomi b

In [41]:
tweet_augmented_cleaned = []

  # Looping setiap kalimat
for sentence in tweet_augmentation:

    sentence = re.sub(r"(pic.twitter.com/[^\s]+)", '', sentence) # remove twitter image link

    sentence = re.sub(r'http[s]?\://\S+', "", sentence) # remove url

    sentence = re.sub(r'\S*.com.*\S', "", sentence)

    sentence = re.sub(r'\S*.co.id.*\S', "", sentence)

    sentence = re.sub(r'([a-z])([A-Z])', r'\1 \2', sentence) # remove kata kata berdempet

    sentence = re.sub(r"#\w+", "", sentence) #remove hashtag

    sentence = sentence.replace('@', '') # remove at@

    sentence = re.sub(r'\b\d+\b', '', sentence) # remove angka berdempet dg kata

    sentence = re.sub('\d', '', sentence) # remove angka

    # sentence = re.sub(r'\b(\w+)\s+\1\b', r'\1', sentence) # Remove reapeating char

    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence) # remove punctuation and special character

    sentence = re.sub(r"\s+", " ", sentence) # remove double space

    sentence = sentence.lower() # case folding

    sentence = sentence.strip()

    tweet_augmented_cleaned.append(sentence)

In [42]:
tweet_augmented_cleaned

['bercakap prabowo indonesia belum dihargai bangsa asing berita ini jelas imitatif buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi bakat dari habib luthfi seharga mercy',
 'di era jokowi ekonomi indonesia semakin baik kedepan lagi menang total debat',
 'bagi sumatera selatan asian games berdampak atas ekonomi langsung diprediksi berhasil triliun indonesia kedepan jokowi hebat',
 'negara kita ngutang buat bngun asas udah dipake masyarakat terus masyarakatnya ngeluh karena negara ngutang setiap keadaan itu pasti ngutang utang bisa dibayar kalo keadaan dapet penghasilan penghasilan negara itu bahwasanya dari pajak',
 'bisikin bapak jokowi cuma mikirin perputaran duit atas golongan pasti esports cuma sebagian kecil dari sekian besar penentu kemajuan perekonomian indonesia belum hal fundamental benar dikatakan prabowo meskipun dengan bahas baru tapi mrpkan aspek asas ekonomi',
 'masa anteng baru ngoceh ajattp jokowi harga mati',
 'bengis balas atas tps kerjasa ekonomi bil

In [43]:
tweet_augmented = pd.DataFrame(tweet_augmented_cleaned, columns=['tweet'])
tweet_augmented

,tweet
0,bercakap prabowo indonesia belum dihargai bang...
1,batuan langka tasbih jokowi bakat dari habib l...
2,di era jokowi ekonomi indonesia semakin baik k...
3,bagi sumatera selatan asian games berdampak at...
4,negara kita ngutang buat bngun asas udah dipak...
...,...
1810,negarawan asal selalu bangga dan bercadang har...
1811,harus ceramah atas damai indonesiaku perekonom...
1812,mari bangkit bangsa dengan mendukung perekonom...
1813,bantu majukan perekonomian bangsa bersama bapa...


In [44]:
df['tweet'] = tweet_augmented['tweet']
df

,sentimen,tweet
0,negatif,bercakap prabowo indonesia belum dihargai bang...
1,netral,batuan langka tasbih jokowi bakat dari habib l...
2,netral,di era jokowi ekonomi indonesia semakin baik k...
3,positif,bagi sumatera selatan asian games berdampak at...
4,negatif,negara kita ngutang buat bngun asas udah dipak...
...,...,...
1810,netral,negarawan asal selalu bangga dan bercadang har...
1811,netral,harus ceramah atas damai indonesiaku perekonom...
1812,netral,mari bangkit bangsa dengan mendukung perekonom...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


In [45]:
df_augmented = df
df_augmented

,sentimen,tweet
0,negatif,bercakap prabowo indonesia belum dihargai bang...
1,netral,batuan langka tasbih jokowi bakat dari habib l...
2,netral,di era jokowi ekonomi indonesia semakin baik k...
3,positif,bagi sumatera selatan asian games berdampak at...
4,negatif,negara kita ngutang buat bngun asas udah dipak...
...,...,...
1810,netral,negarawan asal selalu bangga dan bercadang har...
1811,netral,harus ceramah atas damai indonesiaku perekonom...
1812,netral,mari bangkit bangsa dengan mendukung perekonom...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


In [46]:
df_real = pd.read_csv('D:/personal project/twitter_sentiment_analysis/datasets/tweet.csv')
df_real

,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...,...
1810,1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [47]:
del df_real[df_real.columns[0]]

In [48]:
df_real

,sentimen,tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...
1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [50]:
df_real['tweet'] = tweet_no_stopwords
df_real

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,batuan langka tasbih jokowi hadiah dari habib ...
2,netral,di era jokowi ekonomi indonesia semakin baik m...
3,positif,bagi sumatera selatan asian games berdampak pa...
4,negatif,negara kita ngutang buat bngun infrastruktur u...
...,...,...
1810,netral,negarawan sejati selalu bangga dan mengedepank...
1811,netral,harus ceramah di damai indonesiaku perekonomia...
1812,netral,mari bangun bangsa dengan mendukung perekonomi...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


# Merge Dataset

In [51]:
data_merge = pd.concat([df_real, df_augmented], ignore_index=False)
data_merge

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,batuan langka tasbih jokowi hadiah dari habib ...
2,netral,di era jokowi ekonomi indonesia semakin baik m...
3,positif,bagi sumatera selatan asian games berdampak pa...
4,negatif,negara kita ngutang buat bngun infrastruktur u...
...,...,...
1810,netral,negarawan asal selalu bangga dan bercadang har...
1811,netral,harus ceramah atas damai indonesiaku perekonom...
1812,netral,mari bangkit bangsa dengan mendukung perekonom...
1813,netral,bantu majukan perekonomian bangsa bersama bapa...


In [52]:
data_merge['sentimen'].value_counts()

sentimen
positif    1224
netral     1214
negatif    1192
Name: count, dtype: int64

In [53]:
from sklearn.utils import shuffle

In [54]:
data_augmntd = shuffle(data_merge)
data_augmntd

,sentimen,tweet
320,netral,sederet menteri kabinet kerja daftarpresidenam...
300,netral,buat ciwiciwi mau harga skincare turun tusuk p...
844,netral,jokowi pertumbuhan ekonomi harus disyukuri jan...
476,netral,coba lu ngmng gini waktu jaman lulung dan mtop...
505,netral,boleh ketemuan ya kita berapa nomor whatsapp e...
...,...,...
606,negatif,berharap tiap bulan ada pemiluterus jokowi nya...
1486,negatif,membuat ekonomi maju itu berkembang industri g...
1425,negatif,jokowi nanya ekonomi digital belum penting wad...
1241,negatif,saya gemar mudah aja dulunaikin level ekonomi ...


In [55]:
data_augmntd.to_csv('D:/personal project/twitter_sentiment_analysis/datasets/tweet_aug_shuffle_2.csv')

In [58]:
import pandas as pd

In [60]:
# dataset_augmented = pd.DataFrame(columns=['sentimen', 'tweet'])

# for  real_sentimen, real_tweet, augmntd_sentimen, augmntd_tweet in zip(df_real['sentimen'], df_real['tweet'], df_augmented['sentimen'], df_augmented['tweet']):
#   dataset_augmented = dataset_augmented.append({'sentimen': real_sentimen, 'tweet': real_tweet}, ignore_index=True)
#   dataset_augmented = dataset_augmented.append({'sentimen': augmntd_sentimen, 'tweet': augmntd_tweet}, ignore_index=True)

# dataset_augmented
dataset_augmented = pd.DataFrame(columns=['sentimen', 'tweet'])

for real_sentimen, real_tweet, augmntd_sentimen, augmntd_tweet in zip(df_real['sentimen'], df_real['tweet'], df_augmented['sentimen'], df_augmented['tweet']):
    # Append rows to the DataFrame using concat
    row1 = pd.DataFrame({'sentimen': [real_sentimen], 'tweet': [real_tweet]})
    row2 = pd.DataFrame({'sentimen': [augmntd_sentimen], 'tweet': [augmntd_tweet]})
    
    dataset_augmented = pd.concat([dataset_augmented, row1, row2], ignore_index=True)

dataset_augmented

,sentimen,tweet
0,negatif,kata prabowo indonesia tidak dihargai bangsa a...
1,negatif,bercakap prabowo indonesia belum dihargai bang...
2,netral,batuan langka tasbih jokowi hadiah dari habib ...
3,netral,batuan langka tasbih jokowi bakat dari habib l...
4,netral,di era jokowi ekonomi indonesia semakin baik m...
...,...,...
3625,netral,mari bangkit bangsa dengan mendukung perekonom...
3626,netral,bantu majukan perekonomian bangsa bersama bapa...
3627,netral,bantu majukan perekonomian bangsa bersama bapa...
3628,netral,bapak jokowi mengubah cara pandang ekonomi kin...


In [61]:
dataset_augmented.to_csv('D:/personal project/twitter_sentiment_analysis/datasets/tweet_aug_2.csv')